In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pyaccel import lattice as pyacc_lat
from pyaccel import optics as pyacc_opt
from pyaccel.optics import calc_touschek_energy_acceptance
from mathphys.functions import save_pickle, load_pickle
from apsuite.dynap import DynapXY, DynapEX, PhaseSpace
import pyaccel
import pymodels

from idanalysis import orbcorr as orbcorr

from idanalysis import optics as optics
from idanalysis import IDKickMap
import utils

from run_rk_traj import PHASES, GAPS
from utils import get_idconfig

In [2]:
def create_model_ids():
    """."""
    print('--- model with kickmap ---')
    ids = utils.create_ids_model(phase, gap, rescale_kicks=1)
    # ids = utils.create_ids(phase, gap, rescale_kicks=1)
    model = pymodels.si.create_accelerator(ids=ids)
    model.cavity_on = False
    model.radiation_on = 0
    twiss, *_ = pyaccel.optics.calc_twiss(model, indices='closed')
    print('length : {:.4f} m'.format(model.length))
    print('tunex  : {:.6f}'.format(twiss.mux[-1]/2/np.pi))
    print('tuney  : {:.6f}'.format(twiss.muy[-1]/2/np.pi))
    straight_nr = int(ids[0].subsec[2:4])
    # get knobs and beta locations
    if straight_nr is not None:
        _, knobs, _ = optics.symm_get_knobs(model, straight_nr)
        locs_beta = optics.symm_get_locs_beta(knobs)
    else:
        knobs, locs_beta = None, None

    return model, knobs, locs_beta

In [3]:
def calc_dtune_betabeat(twiss0, twiss1):
    """."""
    dtunex = (twiss1.mux[-1] - twiss0.mux[-1]) / 2 / np.pi
    dtuney = (twiss1.muy[-1] - twiss0.muy[-1]) / 2 / np.pi
    bbeatx = 100 * (twiss1.betax - twiss0.betax) / twiss0.betax
    bbeaty = 100 * (twiss1.betay - twiss0.betay) / twiss0.betay
    bbeatx_rms = np.std(bbeatx)
    bbeaty_rms = np.std(bbeaty)
    bbeatx_absmax = np.max(np.abs(bbeatx))
    bbeaty_absmax = np.max(np.abs(bbeaty))
    return (
      dtunex, dtuney, bbeatx, bbeaty,
      bbeatx_rms, bbeaty_rms, bbeatx_absmax, bbeaty_absmax)

In [4]:
def analysis_uncorrected_perturbation(
        model, idconfig, twiss0=None, plot_flag=True):
    """."""
    config_label = idconfig
    twiss, *_ = pyacc_opt.calc_twiss(model, indices='closed')

    results = calc_dtune_betabeat(twiss0, twiss)
    dtunex, dtuney = results[0], results[1]
    bbeatx, bbeaty = results[2], results[3]
    bbeatx_rms, bbeaty_rms = results[4], results[5]
    bbeatx_absmax, bbeaty_absmax = results[6], results[7]

    if plot_flag:

        print(f'dtunex: {dtunex:+.6f}')
        print(f'dtunex: {dtuney:+.6f}')
        txt = f'bbetax: {bbeatx_rms:04.1f} % rms, '
        txt += f'{bbeatx_absmax:04.1f} % absmax'
        print(txt)
        txt = f'bbetay: {bbeaty_rms:04.1f} % rms, '
        txt += f'{bbeaty_absmax:04.1f} % absmax'
        print(txt)

        labelx = f'X ({bbeatx_rms:.1f} % rms)'
        labely = f'Y ({bbeaty_rms:.1f} % rms)'
        plt.plot(twiss.spos, bbeatx, color='b', alpha=1, label=labelx)
        plt.plot(twiss.spos, bbeaty, color='r', alpha=0.8, label=labely)
        plt.xlabel('spos [m]')
        plt.ylabel('Beta Beat [%]')
        plt.title('Beta Beating from ID - ' + config_label)
        plt.legend()
        plt.grid()
        plt.show()

    return twiss

In [7]:
def plot_beta_beating(twiss0, twiss1, twiss2, twiss3, config_label):
    """."""

    figpath = 'results/phase-organized/{}/gap-{}/'.format(phase, gap)

    # Compare optics between nominal value and uncorrect optics due ID
    results = calc_dtune_betabeat(twiss0, twiss1)
    dtunex, dtuney = results[0], results[1]
    bbeatx, bbeaty = results[2], results[3]
    bbeatx_rms, bbeaty_rms = results[4], results[5]
    bbeatx_absmax, bbeaty_absmax = results[6], results[7]
    print('phase: {} gap: {} \n'.format(phase, gap))
    print('Not symmetrized optics :')
    print(f'dtunex: {dtunex:+.2e}')
    print(f'dtuney: {dtuney:+.2e}')
    print(f'bbetax: {bbeatx_rms:04.2f} % rms, {bbeatx_absmax:04.2f} % absmax')
    print(f'bbetay: {bbeaty_rms:04.2f} % rms, {bbeaty_absmax:04.2f} % absmax')
    print()

    plt.figure(1)
    labelx = f'X ({bbeatx_rms:.2f} % rms)'
    labely = f'Y ({bbeaty_rms:.2f} % rms)'
    plt.plot(twiss0.spos, bbeatx, color='b', alpha=1.0, label=labelx)
    plt.plot(twiss0.spos, bbeaty, color='r', alpha=0.8, label=labely)
    plt.xlabel('spos [m]')
    plt.ylabel('Beta Beating [%]')
    plt.title('Beta Beating from ID - ' + config_label)
    plt.suptitle('Not symmetrized optics')
    plt.xlim(215, 250)
    plt.legend()
    plt.grid()
    plt.savefig(figpath + 'uncorrected-optics', dpi=300)
    plt.close()

    # Compare optics between nominal value and symmetrized optics
    results = calc_dtune_betabeat(twiss0, twiss2)
    dtunex, dtuney = results[0], results[1]
    bbeatx, bbeaty = results[2], results[3]
    bbeatx_rms, bbeaty_rms = results[4], results[5]
    bbeatx_absmax, bbeaty_absmax = results[6], results[7]
    print('symmetrized optics but uncorrect tunes:')
    print(f'dtunex: {dtunex:+.0e}')
    print(f'dtuney: {dtuney:+.0e}')
    print(f'bbetax: {bbeatx_rms:04.2f} % rms, {bbeatx_absmax:04.2f} % absmax')
    print(f'bbetay: {bbeaty_rms:04.2f} % rms, {bbeaty_absmax:04.2f} % absmax')
    print()

    plt.figure(2)
    labelx = f'X ({bbeatx_rms:.2f} % rms)'
    labely = f'Y ({bbeaty_rms:.2f} % rms)'
    plt.plot(twiss0.spos, bbeatx, color='b', alpha=1.0, label=labelx)
    plt.plot(twiss0.spos, bbeaty, color='r', alpha=0.8, label=labely)
    plt.xlabel('spos [m]')
    plt.ylabel('Beta Beating [%]')
    plt.title('Beta Beating from ID - ' + config_label)
    plt.suptitle('Symmetrized optics and uncorrect tunes')
    plt.legend()
    plt.xlim(215, 250)
    plt.grid()
    plt.savefig(figpath + 'corrected-optics', dpi=300)
    plt.close()

    # Compare optics between nominal value and all corrected
    results = calc_dtune_betabeat(twiss0, twiss3)
    dtunex, dtuney = results[0], results[1]
    bbeatx, bbeaty = results[2], results[3]
    bbeatx_rms, bbeaty_rms = results[4], results[5]
    bbeatx_absmax, bbeaty_absmax = results[6], results[7]
    print('symmetrized optics and correct tunes:')
    print(f'dtunex: {dtunex:+.0e}')
    print(f'dtuney: {dtuney:+.0e}')
    print(f'bbetax: {bbeatx_rms:04.2f} % rms, {bbeatx_absmax:04.2f} % absmax')
    print(f'bbetay: {bbeaty_rms:04.2f} % rms, {bbeaty_absmax:04.2f} % absmax')

    plt.figure(3)
    labelx = f'X ({bbeatx_rms:.2f} % rms)'
    labely = f'Y ({bbeaty_rms:.2f} % rms)'
    plt.plot(twiss0.spos, bbeatx, color='b', alpha=1.0, label=labelx)
    plt.plot(twiss0.spos, bbeaty, color='r', alpha=0.8, label=labely)
    plt.xlabel('spos [m]')
    plt.ylabel('Beta Beating [%]')
    plt.title('Beta Beating from ID - ' + config_label)
    plt.suptitle('Symmetrized optics and correct tunes')
    plt.legend()
    plt.grid()
    plt.xlim(215, 250)
    plt.savefig(figpath + 'corrected-optics-tunes', dpi=300)
    plt.close()

In [8]:
def analysis(plot_flag=True):
    """."""
    # select where EPU will be installed
    straight_nr = 10

    # create unperturbed model for reference
    model0 = pymodels.si.create_accelerator()
    model0.cavity_on = False
    model0.radiation_on = 0
    twiss0, *_ = pyacc_opt.calc_twiss(model0, indices='closed')

    # create model with id
    idconfig = utils.get_idconfig(phase, gap)

    model1, knobs, locs_beta = create_model_ids()

    print('local quadrupole fams: ', knobs)
    print('element indices for straight section begin and end: ', locs_beta)

    # calculate nominal twiss
    goal_tunes = np.array(
        [twiss0.mux[-1] / 2 / np.pi, twiss0.muy[-1] / 2 / np.pi])
    goal_beta = np.array(
        [twiss0.betax[locs_beta], twiss0.betay[locs_beta]])
    goal_alpha = np.array(
        [twiss0.alphax[locs_beta], twiss0.alphay[locs_beta]])
    print(goal_beta)

    # correct orbit
    orbcorr.correct_orbit_local(
        model0, model1, 'EPU50', correction_plane='both', plot=False)

    orb_results = orbcorr.correct_orbit_fb(
        model0, model1, 'EPU50', corr_system='SOFB')

    # calculate beta beating and tune delta tunes
    twiss1 = analysis_uncorrected_perturbation(
        model1, idconfig=idconfig, twiss0=twiss0, plot_flag=False)

    # symmetrize optics (local quad fam knobs)
    dk_tot = np.zeros(len(knobs))
    for i in range(7):
        dk = optics.correct_symmetry_withbeta(
            model1, straight_nr, goal_beta, goal_alpha)
        print('iteration #{}, dK: {}'.format(i+1, dk))
        dk_tot += dk
    for i, fam in enumerate(knobs):
        print('{:<9s} dK: {:+9.6f} 1/m²'.format(fam, dk_tot[i]))
    twiss2, *_ = pyacc_opt.calc_twiss(model1, indices='closed')
    print()

    # correct tunes
    tunes = twiss1.mux[-1]/np.pi/2, twiss1.muy[-1]/np.pi/2
    print('init    tunes: {:.9f} {:.9f}'.format(tunes[0], tunes[1]))
    for i in range(2):
        optics.correct_tunes_twoknobs(model1, goal_tunes)
        twiss, *_ = pyacc_opt.calc_twiss(model1)
        tunes = twiss.mux[-1]/np.pi/2, twiss.muy[-1]/np.pi/2
        print('iter #{} tunes: {:.9f} {:.9f}'.format(i+1, tunes[0], tunes[1]))
    print('goal    tunes: {:.9f} {:.9f}'.format(goal_tunes[0], goal_tunes[1]))
    twiss3, *_ = pyacc_opt.calc_twiss(model1, indices='closed')
    print()

    plot_beta_beating(
        model0, twiss0, twiss1, twiss2, twiss3, idconfig)

    return model1

In [11]:
def analysis_dynapt_map(model1, gap, width):

    gap_str = '{:04.1f}'.format(gap).replace('.', 'p')
    model1.radiation_on = 0
    model1.cavity_on = False
    model1.vchamber_on = True

    dynapxy = DynapXY(model1)
    dynapxy.params.x_nrpts = 40
    dynapxy.params.y_nrpts = 20
    dynapxy.params.nrturns = 1*1024
    print(dynapxy)
    dynapxy.do_tracking()
    dynapxy.process_data()
    fig2, *ax = dynapxy.make_figure_diffusion(orders=(1, 2, 3, 4))
    fig2.show()

    fig_path = './results/model/width_{}/dynapt-kickmap-gap{}-width{}-symmetrized.png'.format(width, gap_str, width)
    fig2.savefig(fig_path, dpi=300, format='png')
  

In [12]:
global phase, gap
for phase0 in PHASES:
    phase = phase0
    for gap0 in GAPS:

--- model with kickmap ---
length : 518.3899 m
tunex  : 49.095962
tuney  : 14.151801
local quadrupole fams:  {'QFB_ID': [2252, 2284], 'QDB1_ID': [2246, 2290], 'QDB2_ID': [2258, 2278]}
element indices for straight section begin and end:  [2246, 2291]
[[ 4.10424186  4.10329781]
 [22.3369932  22.3370103 ]]
iteration #1, dK: [-5.14395220e-07 -1.81648256e-05 -3.76781273e-06]
iteration #2, dK: [ 7.95815513e-10 -3.82027721e-09 -6.30026646e-10]
iteration #3, dK: [-5.50268595e-14  4.01443414e-14 -1.76194948e-15]
iteration #4, dK: [ 1.80541926e-14 -1.48726344e-13 -2.68496300e-14]
iteration #5, dK: [7.48594346e-15 4.48325725e-14 1.04159783e-14]
iteration #6, dK: [-2.51513269e-14  1.40202144e-13  2.38499977e-14]
iteration #7, dK: [ 2.39789841e-14 -6.36392629e-14 -8.56913348e-15]
QFB_ID    dK:   -0.0000 1/m² 
QDB1_ID   dK:   -0.0000 1/m² 
QDB2_ID   dK:   -0.0000 1/m² 


init    tunes: 49.095962107 14.151801342
iter #1 tunes: 49.095966634 14.151790358
iter #2 tunes: 49.095966634 14.151790358
goal   

/tmp/ipykernel_2583278/4273130826.py:16: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig2.show()


--- model with kickmap ---
length : 518.3899 m
tunex  : 49.095971
tuney  : 14.153464
local quadrupole fams:  {'QFB_ID': [2252, 2284], 'QDB1_ID': [2246, 2290], 'QDB2_ID': [2258, 2278]}
element indices for straight section begin and end:  [2246, 2291]
[[ 4.10424186  4.10329781]
 [22.3369932  22.3370103 ]]
iteration #1, dK: [ 0.00055395 -0.00305103 -0.00052468]
iteration #2, dK: [ 1.41958401e-05 -7.95417779e-05 -1.37150008e-05]
iteration #3, dK: [ 9.32876990e-09 -5.22788960e-08 -9.01433166e-09]
iteration #4, dK: [ 9.66494565e-14 -2.15732017e-13 -2.67683886e-14]
iteration #5, dK: [-2.64988688e-14  1.03788432e-13  1.64650478e-14]
iteration #6, dK: [ 1.02055343e-14 -1.45473151e-13 -2.79088915e-14]
iteration #7, dK: [-2.87508176e-14  1.54375749e-13  2.64027869e-14]
QFB_ID    dK:   +0.0006 1/m² 
QDB1_ID   dK:   -0.0031 1/m² 
QDB2_ID   dK:   -0.0005 1/m² 


init    tunes: 49.095970948 14.153463627
iter #1 tunes: 49.095966697 14.151791017
iter #2 tunes: 49.095966697 14.151791017
goal    tunes: 4